# R_5.2 Join Electorate to sa1
- so we can take intersection and proportionate data

### Import and preprocess data into correct geographical form

In [1]:
import geopandas as gpd
from pyproj import CRS

/Applications/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf = gpd.read_file('../data/raw/Geography/2021_ELB_region.shp')

In [3]:
# correct geocode for Australia
new_crs = CRS.from_epsg(4283)  

# Reproject the GeoDataFrame to the new CRS
gdf = gdf.to_crs(new_crs)

In [4]:
# remove redundant columns
gdf = gdf[['E_div_numb', 'Elect_div', 'Area_SqKm', 'geometry']]
gdf

,E_div_numb,Elect_div,Area_SqKm,geometry
0,1,Banks,49.47,"POLYGON Z ((151.12967 -33.97363 0.00000, 151.1..."
1,2,Barton,39.65,"POLYGON Z ((151.17424 -33.92497 0.00000, 151.1..."
2,3,Bennelong,58.76,"POLYGON Z ((151.15908 -33.79871 0.00000, 151.1..."
3,4,Berowra,741.64,"POLYGON Z ((151.28480 -33.57221 0.00000, 151.2..."
4,5,Blaxland,61.16,"POLYGON Z ((151.04441 -33.84205 0.00000, 151.0..."
...,...,...,...,...
146,11,O'Connor,1126936.75,"MULTIPOLYGON Z (((129.00205 -30.76778 0.00000,..."
147,12,Pearce,782.75,"POLYGON Z ((115.98284 -31.74189 0.00000, 115.9..."
148,13,Perth,79.92,"POLYGON Z ((115.96432 -31.88769 0.00000, 115.9..."
149,14,Swan,150.89,"POLYGON Z ((116.04102 -31.98350 0.00000, 116.0..."


In [5]:
gdf['own_calculated_area'] = gdf.geometry.area

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/3467881995.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['own_calculated_area'] = gdf.geometry.area


In [6]:
gdf2 = gpd.read_file('../data/raw/Geography/SA1_2021_AUST_GDA2020.shp')

In [7]:
# remove redundant columns
gdf2 = gdf2[['SA1_CODE21', 'GCC_CODE21', 'STE_CODE21', 'AREASQKM21', 'geometry']]

In [8]:
new_crs = CRS.from_epsg(4283)  # Example: WGS 84

# Reproject the GeoDataFrame to the new CRS
gdf2 = gdf2.to_crs(new_crs)

In [9]:
# calculate the intersection of the polygons
intersection = gpd.overlay(gdf, gdf2, how='intersection')

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/589083907.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 16555 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersection = gpd.overlay(gdf, gdf2, how='intersection')


In [10]:
# Reproject the GeoDataFrame to the new CRS
intersection = intersection.to_crs(new_crs)

In [11]:
# find the intersecting area
intersection['intersection_area'] = intersection.geometry.area

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/2089829042.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection['intersection_area'] = intersection.geometry.area


In [12]:
# find the percentage of the overlapped area within the electorate
intersection['percentage_overlap'] = intersection['intersection_area']/intersection['own_calculated_area']

In [13]:
intersection

,E_div_numb,Elect_div,Area_SqKm,own_calculated_area,SA1_CODE21,GCC_CODE21,STE_CODE21,AREASQKM21,geometry,intersection_area,percentage_overlap
0,1,Banks,49.47,0.005169,11901135601,1GSYD,1,1.0114,"MULTIPOLYGON Z (((150.99919 -33.94331 0.00000,...",4.466347e-10,8.640458e-08
1,5,Blaxland,61.16,0.005958,11901135601,1GSYD,1,1.0114,"POLYGON Z ((150.99978 -33.94312 0.00000, 150.9...",9.865423e-05,1.655823e-02
2,1,Banks,49.47,0.005169,11901135602,1GSYD,1,1.0877,"MULTIPOLYGON Z (((151.01044 -33.93979 0.00000,...",3.816341e-10,7.382978e-08
3,5,Blaxland,61.16,0.005958,11901135602,1GSYD,1,1.0877,"POLYGON Z ((151.01044 -33.93979 0.00000, 151.0...",1.060754e-04,1.780381e-02
4,1,Banks,49.47,0.005169,11901135801,1GSYD,1,0.1146,"POLYGON Z ((151.02667 -33.95706 0.00000, 151.0...",1.117232e-05,2.161362e-03
...,...,...,...,...,...,...,...,...,...,...,...
69112,15,Tangney,101.97,0.010736,50704118414,5GPER,5,0.1390,"POLYGON Z ((115.82304 -32.05443 0.00000, 115.8...",1.327154e-05,1.236152e-03
69113,15,Tangney,101.97,0.010736,50704118415,5GPER,5,0.1184,"POLYGON Z ((115.82941 -32.04747 0.00000, 115.8...",1.130331e-05,1.052825e-03
69114,15,Tangney,101.97,0.010736,50704118416,5GPER,5,0.1618,"POLYGON Z ((115.82489 -32.05009 0.00000, 115.8...",1.545413e-05,1.439446e-03
69115,15,Tangney,101.97,0.010736,50704118417,5GPER,5,0.1171,"POLYGON Z ((115.82534 -32.05300 0.00000, 115.8...",1.118998e-05,1.042270e-03


In [14]:
# EDA to see number of meaningful overlap (i.e. consider any below 0.01 as rounding errors, and equally above 0.99)
intersection[(intersection['percentage_overlap'] < 0.99) & (intersection['percentage_overlap'] >0.01)]

,E_div_numb,Elect_div,Area_SqKm,own_calculated_area,SA1_CODE21,GCC_CODE21,STE_CODE21,AREASQKM21,geometry,intersection_area,percentage_overlap
1,5,Blaxland,61.16,0.005958,11901135601,1GSYD,1,1.0114,"POLYGON Z ((150.99978 -33.94312 0.00000, 150.9...",0.000099,0.016558
3,5,Blaxland,61.16,0.005958,11901135602,1GSYD,1,1.0877,"POLYGON Z ((151.01044 -33.93979 0.00000, 151.0...",0.000106,0.017804
9,1,Banks,49.47,0.005169,11901135806,1GSYD,1,0.5865,"POLYGON Z ((151.03721 -33.94064 0.00000, 151.0...",0.000057,0.011066
102,1,Banks,49.47,0.005169,11901165706,1GSYD,1,1.1493,"POLYGON Z ((150.98559 -33.94416 0.00000, 150.9...",0.000112,0.021683
107,5,Blaxland,61.16,0.005958,11901165709,1GSYD,1,1.3429,"POLYGON Z ((150.97517 -33.94574 0.00000, 150.9...",0.000131,0.021979
...,...,...,...,...,...,...,...,...,...,...,...
68177,13,Perth,79.92,0.008081,50401104512,5GPER,5,2.2023,"POLYGON Z ((115.92316 -31.90532 0.00000, 115.9...",0.000210,0.025985
68229,13,Perth,79.92,0.008081,50401104608,5GPER,5,0.9571,"POLYGON Z ((115.89718 -31.94684 0.00000, 115.8...",0.000091,0.011295
68401,14,Swan,150.89,0.015257,50602112001,5GPER,5,4.8502,"POLYGON Z ((115.94942 -31.97360 0.00000, 115.9...",0.000463,0.030332
68589,14,Swan,150.89,0.015257,50605113806,5GPER,5,4.2589,"POLYGON Z ((115.98094 -31.98249 0.00000, 115.9...",0.000406,0.026630


In [15]:
intersection.to_file('../data/curated/GeographicalProportionateData/Elect_SA1.shp')

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_72321/4220631060.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  intersection.to_file('../data/curated/GeographicalProportionateData/Elect_SA1.shp')


In [16]:
intersection[['E_div_numb', 'Elect_div', 'SA1_CODE21', 'percentage_overlap']].to_csv('../data/curated/GeographicalProportionateData/Elect_SA1.csv')